# What is this "mystery" 2nd component?
---
What we want to do here:
* Recreate the BPT and WHAV* diagrams
* Interrogate the properties of the "mystery" 2nd component
* Look for correlations between the properties of the "mystery" 2nd component and other quantities 

Ideas from reading about DIG:
* Is there a correlation between the velocity dispersion offset between components 1 & 2 (—> DIG at higher scaleheights) and SFR surface density? 
* den Brok+2020 found median velocity lags of ~10 km/s in the MAD galaxies. We should try to measure this in the same way in our galaxies, and look at what SFR surface densities these have - kinemetry modelling?
* Eqn. 1 of Belfiore+2022: if the geometry remains the same but we increase the SFR surface density, then this increases the Ha flux, which then proportionally increases the Ha surface brightness. Therefore, in this simple approximation, the ratio between the Ha emission of the DIG and the HII regions remains constant as a function of SFR surface density. Is this what we see in the observations?
* Plot sigma_gas (component 1) vs. sigma_gas (component 2)


In [73]:
%matplotlib widget

In [74]:
import os, sys 
import numpy as np
import pandas as pd 
from astropy.visualization import hist

from spaxelsleuth.loaddata.sami import load_sami_df
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram, vmin_fn, vmax_fn, cmap_fn, label_fn, fname_fn, bpt_labels, bpt_colours, ncomponents_colours, ncomponents_labels, component_colours, component_labels

import matplotlib.pyplot as plt 
plt.ion()
plt.close("all")

In [75]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

In [76]:
savefigs = True
fig_path = os.path.join(os.environ["SAMI_FIG_DIR"], "presentations")

In [77]:
# Load the entire SAMI sample
df_all = load_sami_df(ncomponents="recom",
                  bin_type="default",
                  eline_SNR_min=5,
                  correct_extinction=True,
                  debug=False)

In load_sami_df(): Loading DataFrame...
In load_sami_df(): Finished!


In [78]:
# Limit sample to having high-quality multi-component fits
df = df_all[df_all["Number of components"] == df_all["Number of components (original)"]]
N = df.shape[0]
N_tot = df_all.shape[0]
print(f"{N:d} ({N / N_tot * 100:.2f}%) of spaxels have high-quality components only")

1721358 (61.04%) of spaxels have high-quality components only


In [79]:
# Make a copy containing only SF-like spaxels
df_SF = df.copy()
cond = df_SF["BPT (total)"] == "SF"

In [ ]:
# Limit our sample to 2nd components that are within a few 10s km/s of the narrow component 
v_max = 50
cond &= np.abs(df_SF["v_gas (component 1)"] - df_SF["v_gas (component 2)"]) < v_max

In [ ]:
# Trim 3-component spaxels 
cond &= df_SF["Number of components"] == 2

In [80]:
df_SF = df_SF[cond]

# Print out some statistics

In [81]:
# How many spaxels in total?
print(f"{df_all.shape[0]} spaxels in total from {len(df_all.catid.unique())} galaxies")

2819842 spaxels in total from 2997 galaxies


In [82]:
# How many spaxels have 1, 2, 3 components BEFORE making S/N cuts?
N_tot = df_all.shape[0]
N = df_all[df_all["Number of components (original)"] > 0].shape[0]
print(f"{N:d} ({N / N_tot * 100:.2f}%) spaxels have emission lines")
for nn in range(0, 4):
    N = df_all[df_all["Number of components (original)"] == nn].shape[0]
    print(f"{N:d} ({N / N_tot * 100:.2f}%) spaxels have {nn} ORIGINAL components")
    

1753749 (62.19%) spaxels have emission lines
1066093 (37.81%) spaxels have 0 ORIGINAL components
1649504 (58.50%) spaxels have 1 ORIGINAL components
51023 (1.81%) spaxels have 2 ORIGINAL components
53222 (1.89%) spaxels have 3 ORIGINAL components


In [83]:
# How many spaxels have 1, 2, 3 components AFTER making S/N cuts?
N_tot = df.shape[0]
N_has_lines = df[df["Number of components (original)"] > 0].shape[0]
print(f"{N_has_lines:d} ({N_has_lines / N_tot * 100:.2f}%) spaxels have emission lines")
print("Of these:")
for nn in range(0, 4):
    N = df[df["Number of components (original)"] == nn].shape[0]
    print(f"{N:d} ({N / N_has_lines * 100:.2f}%) spaxels have {nn} ORIGINAL components")
    

655265 (38.07%) spaxels have emission lines
Of these:
1066093 (162.70%) spaxels have 0 ORIGINAL components
606197 (92.51%) spaxels have 1 ORIGINAL components
44400 (6.78%) spaxels have 2 ORIGINAL components
4668 (0.71%) spaxels have 3 ORIGINAL components


In [32]:
# How many spaxels in each BPT category AFTER making S/N cuts?
N_tot = df.shape[0]
N_tot_classified = df[df["BPT (total)"] != "Not classified"].shape[0]
print(f"{N_tot_classified:d} ({N_tot_classified / N_tot * 100:.2f}%) spaxels can be spectrally classified")
print("Of these:")
for cat in ["SF", "Composite", "LINER", "Seyfert", "Ambiguous"]:
    N = df[df["BPT (total)"] == cat].shape[0]
    print("\t" + f"{N:d} ({N / N_tot_classified * 100:.2f}%) spaxels are {cat}-like")
    

253678 (14.74%) spaxels can be spectrally classified
Of these:
	218562 (86.16%) spaxels are SF-like
	16472 (6.49%) spaxels are Composite-like
	3341 (1.32%) spaxels are LINER-like
	2083 (0.82%) spaxels are Seyfert-like
	13220 (5.21%) spaxels are Ambiguous-like


## Stacked bar chart: # of fitted components by spectral classification

In [27]:
counts_0 = []
counts_1 = []
counts_2 = []
counts_3 = []
for bpt in bpt_labels:
    cond = df["BPT (total)"] == bpt
    cond_0 = cond & (df["Number of components"] == 0)
    cond_1 = cond & (df["Number of components"] == 1)
    cond_2 = cond & (df["Number of components"] == 2)
    cond_3 = cond & (df["Number of components"] == 3)
    counts_0.append(df[cond_0].shape[0] / df[cond].shape[0] * 100)
    counts_1.append(df[cond_1].shape[0] / df[cond].shape[0] * 100)
    counts_2.append(df[cond_2].shape[0] / df[cond].shape[0] * 100)
    counts_3.append(df[cond_3].shape[0] / df[cond].shape[0] * 100)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7, 5))
ax.bar(bpt_labels, counts_0, width=0.6, label=ncomponents_labels[0] + " components", color=ncomponents_colours[0])
ax.bar(bpt_labels, counts_1, width=0.6, bottom=counts_0, label=ncomponents_labels[1] + " component", color=ncomponents_colours[1])
ax.bar(bpt_labels, counts_2, width=0.6, bottom=np.array(counts_0) + np.array(counts_1), label=ncomponents_labels[2] + " components", color=ncomponents_colours[2])
ax.bar(bpt_labels, counts_3, width=0.6, bottom=np.array(counts_0) + np.array(counts_1) + np.array(counts_2), label=ncomponents_labels[3] + " components", color=ncomponents_colours[3])
ax.grid()
ax.set_ylim([0, 105])
ax.set_ylabel("Percentage")
ax.set_xlabel("Spectral classification")
ax.legend(loc="lower center", bbox_to_anchor=(0.5, 1.02), ncol=2, fontsize="small")
plt.xticks(rotation=90)

if savefigs:
    fname = os.path.join(fig_path, f"bar_ncomponents_by_spectral_class.pdf")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    print(f"File saved at: {fname}")
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/bar_ncomponents_by_spectral_class.pdf


In [59]:
# BPT diagrams
col_z_list = ["count", "BPT (numeric) (total)", "Number of components", "HALPHA EW (total)"]
col_y = "log O3 (total)"
for col_z in col_z_list:
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)

    # Plot 2D histograms of the subset
    plot2dhistcontours(df, col_x="log N2 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log S2 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log O1 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)
    
    # Decorations
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]
    if col_z == "count":
        [ax.get_children()[0].set_clim([10, 1e3]) for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]
    
    # Save
    if savefigs:
        fig.savefig(os.path.join(fig_path, f"BPT_SAMI_{fname_fn(col_z)}.pdf"), bbox_inches="tight", format="pdf")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Histograms showing the distribution in various quantities in components 1, 2 and 3

In [55]:
for col in ["sigma_gas", "v_gas", "log HALPHA EW"]:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 4))
    for nn in range(1, 4):
        hist(df[f"{col} (component {nn})"], range=(vmin_fn(col), vmax_fn(col)), bins="scott", histtype="step", color=component_colours[nn - 1], ax=ax, normed=True, label=f"Component {nn}")
    ax.set_xlabel(label_fn(col))
    ax.set_ylabel("N (normalised)")
    ax.legend()
    ax.grid()
    
    if savefigs:
        fname = os.path.join(fig_path, f"hist_{fname_fn(col)}_full_sample.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")



/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/hist_sigma_gas_full_sample.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


...

File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/hist_v_gas_full_sample.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/hist_logHaEW_full_sample.pdf


In [57]:
# SF only
for col in ["sigma_gas", "v_gas", "log HALPHA EW"]:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 4))
    for nn in range(1, 4):
        hist(df_SF[f"{col} (component {nn})"], range=(vmin_fn(col), vmax_fn(col)), bins="scott", histtype="step", color=component_colours[nn - 1], ax=ax, normed=True, label=f"Component {nn}")
    ax.set_xlabel(label_fn(col))
    ax.set_ylabel("N (normalised)")
    ax.legend()
    ax.grid()
    
    if savefigs:
        fname = os.path.join(fig_path, f"hist_{fname_fn(col)}_SF_only.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/hist_sigma_gas_SF_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/hist_v_gas_SF_only.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/hist_logHaEW_SF_only.pdf


In [56]:
plt.close("all")

In [50]:
# Histogram showing the offset in velocity from component 1 of components 2 and 3
col = "delta v_gas"
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 4))
hist(df[f"{col} (2/1)"], range=(-300, +300), bins="scott", histtype="step", color="black", ax=ax, normed=False, alpha=0.5, label="Full sample")
hist(df_SF[f"{col} (2/1)"], range=(-300, +300), bins="scott", histtype="step", color="blue", ax=ax, normed=False, label="Star-forming spaxels")
ax.grid()
ax.set_ylabel("N")
ax.set_xlabel(r"$\Delta v_{\rm gas}$ (2/1)")
ax.legend()

if savefigs:
    fname = os.path.join(fig_path, f"delta_v_21_SF_vs_all.pdf")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    print(f"File saved at: {fname}")
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

File saved at: /priv/meggs3/u5708159/SAMI/figs/presentations/delta_v_21_SF_vs_all.pdf


In [72]:
# Halpha EW, SFR, SFR surface density
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["log HALPHA EW (total)", "log SFR (total)", "log SFR surface density (total)"]):
    for nn in range(1, 4):
        cond = df_SF["Number of components"] == nn
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), 100 if col_x.startswith("sigma") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[nn]} component{'s' if nn > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond, col_x].shape[0]),
             color=ncomponents_colours[nn])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if nn > 0 else None
axs[0].axvline(np.log10(3), linestyle="--", color="k")
axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"H$\alpha$-derived quantities (SF spaxels only)", y=0.95)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_HaEW_and_sigma_and_SFR_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/hist_1vs2vs3components_HaEW_and_sigma_and_SFR_SF_only.pdf


In [70]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

# Systematic quantities
for cc, col_x in enumerate(["Bin size (square kpc)", "z_spec", "Inclination i (degrees)"]):
    for nn in range(1, 4):
        cond = df_SF["Number of components"] == nn
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", 
             range=(0, 90) if col_x == "Inclination i (degrees)" else None,
             label=f"{ncomponents_labels[nn]} component{'s' if nn > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[nn])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
# axs[0].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Systematics (SF spaxels only)", y=0.95)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_systematic_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

# HALPHA S/N
for cc, col_x in enumerate(["HALPHA S/N (component 1)", "HALPHA S/N (component 2)", "HALPHA S/N (component 3)"]):
    for nn in range(1, 4):
        cond = df_SF["Number of components"] == nn
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", 
             range=(0, 90) if col_x == "Inclination i (degrees)" else None,
             label=f"{ncomponents_labels[nn]} component{'s' if nn > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[nn])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
# axs[0].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Systematics (SF spaxels only)", y=0.95)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_HALPHA_SNR_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")
    
# Specifically looking at v_grad
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["v_grad (component 1)", "v_grad (component 2)", "v_grad (component 3)"]):
    for nn in range(1, 4):
        cond = df_SF["Number of components"] == nn
        if all(df_SF.loc[cond, col_x].isna()):
            continue
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", range=(0, 150),
             label=f"{ncomponents_labels[nn]} component{'s' if nn > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[nn])
    axs[cc].set_xlabel(label_fn(col_x) + f" (component {cc + 1})")
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"$v_{\rm grad}$ (SF spaxels only)", y=0.95)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_vgrad_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Local quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["D4000", "r/R_e", "sigma_*"]):
    for nn in range(1, 4):
        cond = df_SF["Number of components"] == nn
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], 
             range=(vmin_fn(col_x), 100 if col_x.startswith("sigma_gas") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[nn]} component{'s' if nn > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[nn])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[-1].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Local properties (SF spaxels only)", y=0.95)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_local_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Global quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["R_e (kpc)", "mstar", "log(M/R_e)"]):
    for nn in range(1, 4):
        cond = df_SF["Number of components"] == nn
        hist(df_SF.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), 100 if col_x.startswith("sigma") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[nn]} component{'s' if nn > 1 else ''}" + r" ($N = %d$)" % (df_SF.loc[cond].shape[0]),
             color=ncomponents_colours[nn])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Global properties (SF spaxels only)", y=0.95)

if savefigs:
    fname = os.path.join(fig_path, f"hist_1vs2vs3components_global_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/hist_1vs2vs3components_HaEW_and_sigma_and_SFR_SF_only


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/hist_1vs2vs3components_local_SF_only.pdf


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:125: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saving to /priv/meggs3/u5708159/SAMI/figs/presentations/hist_1vs2vs3components_global_SF_only.pdf


#### Eqn. 1 of Belfiore+2022: if the geometry remains the same but we increase the SFR surface density, then this increases the Ha flux, which then proportionally increases the Ha surface brightness. Therefore, in this simple approximation, the ratio between the Ha emission of the DIG and the HII regions remains constant as a function of SFR surface density. Is this what we see in the observations?


In [22]:
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.7, 0.8])
_ = plot2dhistcontours(df=df_SF, 
                       col_y="log HALPHA EW (component 1)", 
                       col_x="log SFR surface density (total)", 
                       col_z="count", log_z=True,
                       # col_z="log(O/H) + 12 (N2O2_K19/O3O2_K19) (total)",
                       nbins=50, ax=ax)
ax.set_title(r"The mean EW of the 1st component increases significantly" + "\n" + r" as a function of $\Sigma_{\rm SFR}$.")

fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.7, 0.8])
_ = plot2dhistcontours(df=df_SF, 
                       col_y="log HALPHA EW (component 2)", 
                       col_x="log SFR surface density (total)", 
                       col_z="count", log_z=True,
                       # col_z="log(O/H) + 12 (N2O2_K19/O3O2_K19) (total)",
                       nbins=50, ax=ax)
ax.set_title(r"The mean EW of the 2nd component remains approximately" + "\n" + r"constant as a function of $\Sigma_{\rm SFR}$.")


Text(0.5, 1.0, 'The mean EW of the 2nd component remains approximately\nconstant as a function of $\\Sigma_{\\rm SFR}$.')

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f28eaca6170> (for post_execute):


FileNotFoundError: missing font metrics file: cmr12

FileNotFoundError: missing font metrics file: cmr12

<Figure size 432x288 with 2 Axes>

FileNotFoundError: missing font metrics file: cmr12

<Figure size 432x288 with 2 Axes>

In [68]:
df_SF["Delta HALPHA EW (2/1)"] = - 1 * df_SF["Delta HALPHA EW (1/2)"]
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.7, 0.8])
_ = plot2dhistcontours(df=df_SF, 
                   col_y="Delta HALPHA EW (2/1)", 
                   col_x="log SFR surface density (total)", 
                   ymin=-2, ymax=+2,
                   col_z="count", log_z=True,
                   nbins=50, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Is there a correlation between the velocity dispersion offset between components 1 & 2 (—> DIG at higher scaleheights) and SFR surface density?

In [72]:
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.7, 0.8])
plot2dhistcontours(df=df_SF, 
                   col_x="log sigma_gas (component 1)", 
                   col_y="log sigma_gas (component 2)", 
                   col_z="log SFR surface density (total)", 
                   xmax=2.1, ymax=2.75, vmin=-3, vmax=-0.5, nbins=50, ax=ax)
ax.set_xlabel(ax.get_xlabel() + " (component 1)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([1.0, 2.1], [2.0, 3.1], "k:")
ax.plot([1.0, 2.1], [1.5, 2.6], "k--")
ax.plot([1.0, 2.1], [1.0, 2.1], "k-")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.7, 0.8])
plot2dhistcontours(df=df_SF, 
                   col_x="log SFR surface density (total)", 
                   col_y="log sigma_gas (component 2)", 
                   col_z="count", log_z=True, 
                   nbins=50, ax=ax)
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([1.0, 2.1], [2.0, 3.1], "k:")
ax.plot([1.0, 2.1], [1.5, 2.6], "k--")
ax.plot([1.0, 2.1], [1.0, 2.1], "k-")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## What happens if we plot the gas velocity of component 2 against that of component 1?

In [7]:
# Experimenting: 2D histogram 
fig = plot2dhistcontours(df_SF,
                         col_x="v_gas (component 1)",
                         col_y="v_gas (component 2)",
                         col_z="count", log_z=True, 
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         nbins=50)
ax = fig.get_axes()[0]
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 1)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# Experimenting: 2D histogram 
fig = plot2dhistcontours(df_SF,
                         col_x="v_gas (component 1)",
                         col_y="v_gas (component 2)",
                         col_z="v_gas - v_* (component 2)",
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         vmin=-100, vmax=+100,
                         nbins=50)
ax = fig.get_axes()[0]
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 1)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
for nn in range(1, 4):
    fig = plot2dhistcontours(df_SF,
                             col_x="v_*",
                             col_y=f"v_gas (component {nn})",
                             col_z="count", log_z=True, 
                             # col_z=f"sigma_*",
                             xmin=-300, xmax=300, 
                             ymin=-300, ymax=300, 
                             nbins=50)
    ax = fig.get_axes()[0]
    ax.grid()
    ax.set_ylabel(ax.get_ylabel() + f" (component {nn})")
    ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:472: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Nice examples of galaxies with "hysteresis" 
* 301346
* 619095
* 41144
* 460374
* 84107 (not hysteresis, but rather no velocity offset from component 1 at all)
* 272990


In [26]:
# Re-make the above plots but overlaid with points from individual galaxies 
fig = plot2dhistcontours(df_SF,
                         col_x="v_gas (component 1)",
                         col_y="v_gas (component 2)",
                         col_z="count", log_z=False, cmap="gray_r", alpha=0.4,
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         plot_colorbar=False,
                         nbins=50)
ax = fig.get_axes()[0]
 
gal = 460374
df_gal = df_all[df_all["catid"] == gal]
_ = plot2dscatter(df_gal, col_x="v_gas (component 1)", col_y="v_gas (component 2)", col_z="sigma_gas (component 2)",
                  xmin=-300, xmax=300, ymin=-300, ymax=300, vmax=150,
                  plot_colorbar=True,
                  ax=ax)

# Decorations 
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 1)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")
ax.plot([-300, 300], [0, 0], color="k")
ax.plot([0, 0], [-300, 300], color="k")

# SDSS image
from spaxelsleuth.plotting.sdssimg import plot_sdss_image
plot_sdss_image(df_gal)

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:472: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/sdssimg.py:164: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize=figsize, subplot_kw={"projection": wcs})


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## How to tell whether the 2nd component is due to residuals in the stellar template fitting?

In [10]:
# Experimenting: 2D histogram 
fig = plot2dhistcontours(df_SF,
                         col_x="v_gas - v_* (component 2)",
                         col_y="sigma_gas - sigma_* (component 2)",
                         col_z="count", log_z=True,
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         nbins=100)
ax = fig.get_axes()[0]
ax.grid()
ax.set_xlabel(ax.get_xlabel() + " (component 2)")
ax.set_ylabel(ax.get_ylabel() + " (component 2)")
ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## BPT diagrams
---

In [20]:
col_z = "log sigma_gas"
for nn in range(1, 4):
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)

    # Plot 2D histograms of the subset
    plot2dhistcontours(df, col_x="log N2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=np.log10(20), vmax=np.log10(150), cmap="Spectral_r", ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log S2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=np.log10(20), vmax=np.log10(150), cmap="Spectral_r", ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log O1 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=np.log10(20), vmax=np.log10(150), cmap="Spectral_r", ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"BPT_SAMI_{fname_fn(col_z)}_component{nn}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")

col_z = "sigma_gas - sigma_*"
for nn in range(1, 4):
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)

    # Plot 2D histograms of the subset
    plot2dhistcontours(df, col_x="log N2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=-200, vmax=+200, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log S2 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=-200, vmax=+200, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df, col_x="log O1 (total)", col_y="log O3 (total)", col_z=f"{col_z} (component {nn})", log_z=False, vmin=-200, vmax=+200, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"BPT_SAMI_{fname_fn(col_z)}_component{nn}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## WHAV* diagram
---

In [15]:
col_z_list = ["count", "BPT (numeric) (total)", "v_grad", "r/R_e"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for nn in range(1, 4):
        plot2dhistcontours(df, col_x=f"sigma_gas - sigma_* (component {nn})",
                           col_y=f"log HALPHA EW (component {nn})",
                           col_z=f"{col_z} (component {nn})" if f"{col_z} (component {nn})" in df else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[nn - 1], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if nn == 3 - 1 else False)
        # Decorations
        axs[nn - 1].grid()
        axs[nn - 1].set_ylabel("") if nn > 0 else None
        axs[nn - 1].set_yticklabels([]) if nn > 0 else None
        axs[nn - 1].text(s=f"Component {nn}", x=0.05, y=0.95, transform=axs[nn - 1].transAxes, verticalalignment="top")

    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"WHAV_nostar_SAMI_{fname_fn(col_z)}.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")        
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …